In [1]:
import os
import pandas as pd
import difflib

In [2]:
actual_labels_path = os.getcwd() + '/image_phone_data/real_data/actual_image_labels.txt'

In [3]:
model_labels_path = os.getcwd() + '/image_phone_data/real_data/0beams_3e4lr_squaremakeborder.txt'

In [4]:
df_rows = []

with open(model_labels_path, "r") as model_labels_fn, open(actual_labels_path, "r") as actual_labels_fn:
    for line1, line2 in zip(model_labels_fn, actual_labels_fn):
        df_rows.append((line1.strip(), line2.strip()))

df = pd.DataFrame(df_rows, columns=['prediction', 'actual'])
df

,prediction,actual
0,9788549999,9788549999
1,646-543-9168,646-543-9168
2,718-633-3688,718-633-3688
3,6465439168,6465439168
4,9293311013,9293371013
...,...,...
83,91763 790 732521133,"917 673 7380, 732 522 1373"
84,7173339997,7173339997
85,646-543-9168,646-543-9168
86,6469446888,6469446888


In [5]:
# Remove "anomaly" labels that contain two or more phone numbers

df = df[df["actual"].apply(len) < 18]
df

,prediction,actual
0,9788549999,9788549999
1,646-543-9168,646-543-9168
2,718-633-3688,718-633-3688
3,6465439168,6465439168
4,9293311013,9293371013
...,...,...
82,6466997731,6466997731
84,7173339997,7173339997
85,646-543-9168,646-543-9168
86,6469446888,6469446888


In [14]:
from jiwer import cer

df.loc[:, "cer"] = df.apply(lambda row: cer(row["prediction"], row["actual"]), axis=1)
mean_cer = df["cer"].mean()

print(f"Mean Character Error Rate: {mean_cer}")
print(f"Overall accuracy: {1 - mean_cer}")


Mean Character Error Rate: 0.03451589061345159
Overall accuracy: 0.9654841093865484


In [16]:
sorted_df = df.sort_values(by="cer")
sorted_df.tail()

,prediction,actual,cer
64,919720505150,9197205050,0.166667
44,6481090478,6467090478,0.200000
37,9292549346,9292649345,0.200000
56,646237150,6462571150,0.222222
69,91722941,9172329451,0.250000


In [13]:
unequal_length_predictions = df[df["prediction"].apply(len) != df["actual"].apply(len)]
unequal_length_predictions

,prediction,actual
31,917485173,9174851713
36,92926485555,9292648555
40,347496827,3474096827
46,64655439168,6465439168
56,646237150,6462571150
58,347369537,3473695377
64,919720505150,9197205050
69,91722941,9172329451
74,34754222276,3475422276
77,646257150,6462571150


In [17]:
equal_length_predictions = df[df["prediction"].apply(len) == df["actual"].apply(len)]
equal_length_predictions

,prediction,actual
0,9788549999,9788549999
1,646-543-9168,646-543-9168
2,718-633-3688,718-633-3688
3,6465439168,6465439168
4,9293311013,9293371013
...,...,...
82,6466997731,6466997731
84,7173339997,7173339997
85,646-543-9168,646-543-9168
86,6469446888,6469446888


In [33]:
def count_different_characters(str1, str2):
    matcher = difflib.SequenceMatcher(None, str1, str2)
    num_matches = sum(block.size for block in matcher.get_matching_blocks())
    num_error = max(len(str1), len(str2)) - num_matches
    return num_error

equal_length_predictions['num_character_error'] = df.apply(lambda row: count_different_characters(str(row['prediction']), str(row['actual'])), axis=1)
equal_length_predictions['len(actual)'] = df.apply(lambda row: len(row['actual']), axis=1)
equal_length_predictions

/tmp/ipykernel_5022/3691075879.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  equal_length_predictions['num_character_error'] = df.apply(lambda row: count_different_characters(str(row['prediction']), str(row['actual'])), axis=1)
/tmp/ipykernel_5022/3691075879.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  equal_length_predictions['len(actual)'] = df.apply(lambda row: len(row['actual']), axis=1)


,prediction,actual,num_character_error,len(actual)
0,9788549999,9788549999,0,10
1,646-543-9168,646-543-9168,0,12
2,718-633-3688,718-633-3688,0,12
3,6465439168,6465439168,0,10
4,9293311013,9293371013,1,10
...,...,...,...,...
82,6466997731,6466997731,0,10
84,7173339997,7173339997,0,10
85,646-543-9168,646-543-9168,0,12
86,6469446888,6469446888,0,10


In [37]:
total_characters = equal_length_predictions["len(actual)"].sum()
total_character_error = equal_length_predictions["num_character_error"].sum()
accuracy = (total_characters - total_character_error) / total_characters
accuracy

0.9802110817941952